###  Fine-tuning the top layers of a pre-trained network 

In this last part, we shall fine tune the top layers of the previous model, in order to adapt the model for the task of recognizing cats and dogs.

As we know, the base layers of a CNN learn rudimentary features such as edges and corners, and the top most layers learn to detect high level features such as legs, face, ears etc. The aim is to fine-tune the top layers so that the CNN is able to learn high level features for just cats and dogs, rather than high level features of all the classes that the original VGG-16 was trained to learn.

### Data loading

In [29]:
##This notebook is built around using tensorflow as the backend for keras
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

/bin/sh: 1: python: not found


In [30]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import scipy
import pylab as pl
import matplotlib.cm as cm
%matplotlib inline

In [31]:
# dimensions of our images.
img_width, img_height = 50, 50

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

## Fine-tuning the top layers of a pre-trained network

Start by instantiating the VGG base and loading its weights.

In [32]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_1")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_2")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_1")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_2")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to

In [33]:
import h5py
f = h5py.File('models/vgg/vgg16_weights.h5')
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]

    #if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
    if weights:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))

    layer.set_weights(weights)

f.close()

Build a classifier model to put on top of the convolutional model. For the fine tuning, we start with a fully trained-classifer. We will use the weights from the earlier model. And then we will add this model on top of the convolutional base.

In [34]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.load_weights('models/bottleneck_40_epochs.h5')

model_vgg.add(top_model)

For fine turning, we only want to train a few layers.  This line will set the first 25 layers (up to the conv block) to non-trainable.

In [35]:
for layer in model_vgg.layers[:25]:
    layer.trainable = False

In [36]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model_vgg.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [37]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

Found 1402 images belonging to 2 classes.
Found 101 images belonging to 2 classes.


In [38]:
nb_epoch = 20
# fine-tune the model
model_vgg.fit_generator(
        train_generator,
        samples_per_epoch=1400,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=43, epochs=20, validation_steps=100)`
  


Epoch 1/20
43/43 [==============================] - 251s 6s/step - loss: 1.1474 - acc: 0.7589 - val_loss: 0.4342 - val_acc: 0.8317
Epoch 2/20
43/43 [==============================] - 246s 6s/step - loss: 0.5375 - acc: 0.8136 - val_loss: 0.3861 - val_acc: 0.8317
Epoch 3/20
43/43 [==============================] - 246s 6s/step - loss: 0.4373 - acc: 0.8428 - val_loss: 0.3859 - val_acc: 0.8614
Epoch 4/20
43/43 [==============================] - 246s 6s/step - loss: 0.3519 - acc: 0.8634 - val_loss: 0.3482 - val_acc: 0.8812
Epoch 5/20
43/43 [==============================] - 247s 6s/step - loss: 0.2979 - acc: 0.8762 - val_loss: 0.3666 - val_acc: 0.8614
Epoch 6/20
43/43 [==============================] - 247s 6s/step - loss: 0.2941 - acc: 0.8863 - val_loss: 0.3479 - val_acc: 0.8614
Epoch 7/20
43/43 [==============================] - 246s 6s/step - loss: 0.2542 - acc: 0.8961 - val_loss: 0.3495 - val_acc: 0.8713
Epoch 8/20
43/43 [==============================] - 246s 6s/step - loss: 0.2172 - a

In [39]:
model_vgg.save_weights('models/finetuning_20epochs_vgg.h5')

In [40]:
model_vgg.load_weights('models/finetuning_20epochs_vgg.h5')

Computing loss and accuracy :

In [41]:
model_vgg.evaluate_generator(validation_generator, 100)

[0.3858799598902224, 0.8712871291849873]

### Evaluating on validation set

## Accuracy = ~87 %

As we can see, just a little fine-tuning yields an accuracy that of 87%! This is well over the baseline accuracy of ~ 80%, which is also stated in this [paper](http://xenon.stanford.edu/~pgolle/papers/dogcat.pdf).